In [ ]:
%env OGDF_BUILD_DIR=~/Bachelorarbeit/OGDF/build-debug

from ogdf_python_widget import ogdf, cppinclude
from ogdf_python_widget.widget import Widget
import ogdf_python_widget

cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")

cppinclude("ogdf/planarity/PlanarizationLayout.h")
cppinclude("ogdf/planarity/SubgraphPlanarizer.h")
cppinclude("ogdf/planarity/PlanarSubgraphFast.h")
cppinclude("ogdf/orthogonal/OrthoLayout.h")
cppinclude("ogdf/planarity/VariableEmbeddingInserter.h")
cppinclude("ogdf/planarity/EmbedderMinDepthMaxFaceLayers.h")
####################################################################################

G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.randomPlanarTriconnectedGraph(G, 20, 40)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()

SL = ogdf.SugiyamaLayout()
SL.call(GA)

####################################################################################
G2 = ogdf.Graph()
ogdf.setSeed(2)
ogdf.randomPlanarTriconnectedGraph(G2, 20, 40)

GA3 = ogdf.GraphAttributes(G2, ogdf.GraphAttributes.all)
for n in G2.nodes:
    GA3.label[n] = "N%s" % n.index()
    GA3.shape[n] = ogdf.Shape.Ellipse
####################################################################################

GA2 = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

for n in G.nodes:
    GA2.label[n] = "N%s" % n.index()
    

pl = ogdf.PlanarizationLayout()

crossMin = ogdf.SubgraphPlanarizer()

ps = ogdf.PlanarSubgraphFast[int]()
ps.__python_owns__ = False
ps.runs(100)
ves = ogdf.VariableEmbeddingInserter()
ves.__python_owns__ = False
ves.removeReinsert(ogdf.RemoveReinsertType.All)

crossMin.setSubgraph(ps)
crossMin.setInserter(ves)

emb = ogdf.EmbedderMinDepthMaxFaceLayers()
emb.__python_owns__ = False
pl.setEmbedder(emb)

ol = ogdf.OrthoLayout()
ol.__python_owns__ = False
ol.separation(20.0)
ol.cOverhang(0.4)
pl.setPlanarLayouter(ol)

#pl.call(GA2)


####################################################################################



w = Widget(GA, True)

movingLinkIds = []
selectedNode = None
selectedLink = None

defaultNodeColor = ogdf.Color(255, 255, 255)
selectedColor = ogdf.Color(150, 231, 255)

def changeNodeColor(node, color):
    w.graph_attributes.fillColor[node] = color
    w.update_node(node, False)

def clickNode(node, alt, ctrl):
    global w, selectedNode
    if not alt and not ctrl:
        if selectedNode is not None:
            changeNodeColor(selectedNode, defaultNodeColor)
        changeNodeColor(node, selectedColor)
        update_selected_node(node)
    if alt:
        #deletes node when clicking on it while holding alt key
        w.graph_attributes.constGraph().delNode(node)
    if ctrl:
        #adds links when holding ctrl
        if selectedNode is not None:
            w.graph_attributes.constGraph().newEdge(selectedNode, node)
            changeNodeColor(selectedNode, defaultNodeColor)
            update_selected_node(None)
            
def nodeMoved(node, x, y):
    if(selectedNode is node):
        update_node_texts()
    
def clickLink(link, alt, ctrl):
    global w, movingLinkIds, selectedLink
    if alt and not ctrl:
        #deletes link when clicking on it while holding alt key
        w.graph_attributes.constGraph().delEdge(link)
    if ctrl and not alt:
        #ctrl toggles link movement
        if link.index() in movingLinkIds:
            w.remove_bend_mover_for_id(link.index())
            movingLinkIds.remove(link.index())
        else:
            w.move_link(link)
            movingLinkIds.append(link.index())
    if not alt and not ctrl:
        #select link
        if selectedLink is not None:
            #unselect currently selected
            w.graph_attributes.strokeColor[selectedLink] = ogdf.Color(0, 0, 0)
            w.update_link(selectedLink)
        w.graph_attributes.strokeColor[link] = ogdf.Color(150, 231, 255)
        w.update_link(link)
        update_selected_link(link)
            
        
def clickSvg(x, y, alt, ctrl, backgroundClicked):
    global w, selectedNode, selectedLink
    if backgroundClicked and not ctrl and not alt:
        #unselect node
        if selectedNode is not None:
            w.graph_attributes.fillColor[selectedNode] = ogdf.Color(255, 255, 255)
            w.update_node(selectedNode, False)
            update_selected_node(None)
        #unselect link
        if selectedLink is not None:
            w.graph_attributes.strokeColor[selectedLink] = ogdf.Color(0, 0, 0)
            w.update_link(selectedLink)
            update_selected_link(None)
    if backgroundClicked and ctrl:
        #adds node when clicking svg and holding ctrl
        coords = w.svgcoords_to_graphcoords(x,y)
        n = w.graph_attributes.constGraph().newNode()
        w.graph_attributes.x[n] = coords['x']
        w.graph_attributes.y[n] = coords['y']
        w.graph_attributes.label[n] = "N%s" % n.index()
        w.update_node(n, False)
        
def clickBend(link, bendIndex, altKey):
    bends = w.graph_attributes.bends[link]
    if altKey:
        bends.removeFirst(bends.get(bendIndex).__deref__())
        w.update_link(link)
        
w.on_link_click_callback = clickLink
w.on_node_click_callback = clickNode
w.on_svg_click_callback = clickSvg
w.on_node_moved_callback = nodeMoved
w.on_bend_clicked_callback = clickBend
        
def update_selected_link(link):
    global selectedLink
    selectedLink = link
    update_link_texts()
    
def update_selected_node(node):
    global selectedNode
    selectedNode = node
    update_node_texts()


import ipywidgets as widgets
from ipywidgets import HBox, VBox

##################################################################################

addNodeMoveToggle = widgets.Checkbox(
    value=False,
    description='move Nodes',
    disabled=False,
    indent=False
)

def on_addNodeMoveToggle(change):
    global w, addNodeMoveToggle
    w.enable_node_movement(addNodeMoveToggle.value)
addNodeMoveToggle.observe(on_addNodeMoveToggle, 'value')

##################################################################################

def create_text_widget(name, disabled = False, placeholder='-'):
    return widgets.Text(
    value='',
    placeholder=placeholder,
    description=name,
    disabled=disabled
)

################################################################################## 

def update_link_texts():
    global link_id_text, selectedLink
    if selectedLink is not None:
        link_id_text.value = str(selectedLink.index())
        link_source_id_text.value = str(selectedLink.source().index())
        link_target_id_text.value = str(selectedLink.target().index())
    else:
        link_id_text.value = ''
        link_source_id_text.value = ''
        link_target_id_text.value = ''

link_id_text = create_text_widget('linkId:', True)
link_source_id_text = create_text_widget('sourceId:', True)
link_target_id_text = create_text_widget('targetId:', True)

##################################################################################

def update_node_texts():
    global node_id_text, selectedNode
    if selectedNode is not None:
        node_id_text.value = str(selectedNode.index())
        node_label_text.value = str(w.graph_attributes.label[selectedNode])
        node_x_text.value = str(int(w.graph_attributes.x[selectedNode]))
        node_y_text.value = str(int(w.graph_attributes.y[selectedNode]))
    else:
        node_id_text.value = ''
        node_label_text.value = ''
        node_x_text.value = ''
        node_y_text.value = ''
        

node_id_text = create_text_widget('nodeId:', True)
node_label_text = create_text_widget('label:')
node_x_text = create_text_widget('x:')
node_y_text = create_text_widget('y:')

##################################################################################

def on_save_button_clicked(b):
    if selectedNode is not None:
        w.graph_attributes.label[selectedNode] = node_label_text.value
        if node_x_text.value.lstrip("-").isdigit() and node_y_text.value.lstrip("-").isdigit():
            w.graph_attributes.x[selectedNode] = int(node_x_text.value)
            w.graph_attributes.y[selectedNode] = int(node_y_text.value)
        w.update_node(selectedNode)
        for edge in selectedNode.adjEntries:
            w.update_link(edge.theEdge())
        
save_button = widgets.Button(description="Save")
save_button.on_click(on_save_button_clicked)

##################################################################################

def on_download_button_clicked(b):
    if not svg_file_name_text.value:
        w.download_svg()
    else:
        w.download_svg(svg_file_name_text.value)

download_button = widgets.Button(description="Download SVG")
svg_file_name_text = create_text_widget('SVG-name:', placeholder='leave empty for datetime-name')

download_button.on_click(on_download_button_clicked)

##################################################################################

hbox = HBox([addNodeMoveToggle])

link_texts_vbox = VBox([link_id_text, link_source_id_text, link_target_id_text])
node_texts_vbox = VBox([node_id_text, node_label_text, node_x_text, node_y_text, save_button])
all_texts_hbox = HBox([link_texts_vbox, node_texts_vbox])
download_texts_hbox = HBox([svg_file_name_text, download_button])

vbox = VBox([hbox, download_texts_hbox, all_texts_hbox, w])

vbox

In [ ]:
w.download_svg()

In [ ]:
GA

In [ ]:
G

In [ ]:
w.start_force_directed()

In [ ]:
w.start_force_directed(fix_start_position=False)

In [ ]:
w.stop_force_directed()

In [ ]:
w.send({'code': 'test'})

In [ ]:
w.refresh_graph()

In [ ]:
w.update_graph_attributes(GA2)

In [ ]:
w.set_graph_attributes(GA3)

In [ ]:
w.update_graph_attributes(GA)

In [ ]:
w.click_thickness = 8

In [ ]:
n = G.newNode()
GA.label[n] = "N%s" % n.index()
w.update_node(n)
e = G.newEdge(G.nodes[2], n)
e2 = G.newEdge(G.nodes[1], n)

In [ ]:
GA.x[n] = -1342
GA.y[n] = -1311
for edge in n.adjEntries:
    w.update_link(edge.theEdge())
w.update_node(n)

In [ ]:
w.send({'code': 'test'})
w.zoom = 1
w.x_pos = 0
w.y_pos = 0

In [ ]:
w.export_graph()

In [ ]:
w.width = 500